In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.3.1%2Bcu118-cp38-cp38-win_amd64.whl (2673.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.18.1%2Bcu118-cp38-cp38-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.3.1%2Bcu118-cp38-cp38-win_amd64.whl (4.0 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached https://download.pytorch.org/whl/mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
     ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
     --------- ----------------------------- 3.6/14.9 MB 117.3 MB/s eta 0:00:01
 

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install tiktoken  openl3  soundfile

  Using cached tiktoken-0.7.0-cp38-cp38-win_amd64.whl.metadata (6.8 kB)
  Using cached openl3-0.4.2-py2.py3-none-any.whl
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached regex-2024.5.15-cp38-cp38-win_amd64.whl.metadata (41 kB)
  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl.metadata (58 kB)
  Using cached kapre-0.3.7-py3-none-any.whl
  Using cached resampy-0.2.2-py3-none-any.whl
  Using cached h5py-3.11.0-cp38-cp38-win_amd64.whl.metadata (2.5 kB)
  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached scikit_image-0.21.0-cp38-cp38-win_amd64.whl.metadata (14 kB)
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached numba-0.58.1-cp3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu118 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [37]:
import openl3
import soundfile

class AudioEncoder():
    def __init__(self) -> None:
        self.audio_encoder_model = openl3.models.load_audio_embedding_model(input_repr="mel256", content_type="music",
                                                                            embedding_size=6144)
    def encode(self, audio_path):
        audio, sr = soundfile.read(audio_path)
        embeddings,timestamps = openl3.get_audio_embedding(audio, sr, model=self.audio_encoder_model)
        return torch.tensor(embeddings, dtype=torch.float32)
    


In [38]:
import tiktoken

class TextTokenizer: 
    def __init__(self) -> None:
        self.encodermodel = tiktoken.get_encoding("cl100k_base")
        
    def encode(self , text):
        tokentext =self.encodermodel.encode(text)
        return torch.tensor(tokentext, dtype=torch.float32)


In [39]:
class JointEmbeddingModel(nn.Module):
    def __init__(self, audio_embed_dim, text_embed_dim, shared_embed_dim):
        super(JointEmbeddingModel, self).__init__()
        self.audio_fc = nn.Linear(audio_embed_dim, shared_embed_dim)
        self.text_fc = nn.Linear(text_embed_dim, shared_embed_dim)
    
    def forward(self, audio_embedding, text_embedding):
        audio_shared = self.audio_fc(audio_embedding)
        text_shared = self.text_fc(text_embedding)
        return audio_shared, text_shared

    def compute_loss(self, audio_shared, text_shared):
        
        positive_pair_dist = F.pairwise_distance(audio_shared, text_shared)
        negative_pair_dist = F.pairwise_distance(audio_shared, torch.roll(text_shared, shifts=1, dims=0))
        margin = 1.0  
        loss = torch.mean(positive_pair_dist) + torch.mean(F.relu(margin - negative_pair_dist))
        return loss


In [40]:
text_tokenizer = TextTokenizer()
audio_encoder = AudioEncoder()

In [41]:
sample_text = "Hello My name is Dhruv"
sample_audio_path = "test_file.wav"

In [42]:
text_embedding = text_tokenizer.encode(sample_text)
audio_embedding = audio_encoder.encode(sample_audio_path)
print("Text Embedding Shape:", text_embedding.shape)
print("Audio Embedding Shape:", audio_embedding.shape)

54/54 [==============================] - 104s 2s/step
Text Embedding Shape: torch.Size([7])
Audio Embedding Shape: torch.Size([1701, 6144])


In [43]:
audio_embed_dim = audio_embedding.shape[-1]  
text_embed_dim = text_embedding.shape[-1]    
shared_embed_dim = 256

In [44]:
model = JointEmbeddingModel(audio_embed_dim, text_embed_dim, shared_embed_dim)
print(model)

JointEmbeddingModel(
  (audio_fc): Linear(in_features=6144, out_features=256, bias=True)
  (text_fc): Linear(in_features=7, out_features=256, bias=True)
)


In [45]:
audio_shared, text_shared = model(audio_embedding, text_embedding)
loss = model.compute_loss(audio_shared, text_shared)

In [46]:
print("Audio Shared Embedding Shape:", audio_shared.shape)
print("Text Shared Embedding Shape:", text_shared.shape)
print("Loss:", loss.item())

Audio Shared Embedding Shape: torch.Size([1701, 256])
Text Shared Embedding Shape: torch.Size([256])
Loss: 153423.375
